In [1]:
from utils.class_patient import Patients


patients = Patients()
len(patients)

1213

In [2]:
# fill measures whose null represent false value

nullableMeasures = [
    "dka_type",
    "macroangiopathy",
    "microangiopathy",
    "mechanical_ventilation",
    "use_NaHCO3",
    "history_aci",
    "history_ami",
    "congestive_heart_failure",
    "liver_disease",
    "ckd_stage",
    "malignant_cancer",
    "hypertension",
    "uti",
    "chronic_pulmonary_disease",
]

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [3]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])        
        print(measure, count)

pco2 917
ph 954
po2 917
albumin 406
hba1c 326
lymphocyte 446
height 415
urine-ketone 294
crp 19


In [4]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1209

## split patients

In [5]:
splitedPatients = patients.split(5, 27)

len(splitedPatients[0])

242

In [6]:
def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]
        
        trainPatientsList = splitedPatients[:i] + splitedPatients[i+1:]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem
        
        yield trainPatients, testPatients

In [7]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

967 242
967 242
967 242
967 242
968 241


In [8]:
train, test = trainTest().send(None)

In [9]:
from pandas import Timedelta


train.getMeasuresBetween(Timedelta(hours=0), Timedelta(hours=1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,race,rr,saps2,sbp,scr,sofa,use_NaHCO3,uti,wbc,weight
0,10924444,20065893,31555593,False,30.0,23,261.0,7.0,17.0,9.5,...,WHITE,32.500000,19,133.0,0.9,0,0.0,0,22.9,58.8
0,11236474,20146147,34965843,True,NaN,40,NaN,NaN,NaN,NaN,...,BLACK/AFRICAN AMERICAN,15.000000,21,136.0,NaN,3,0.0,1,16.1,52.9
0,18135181,20173681,30217596,True,14.0,54,304.0,23.0,15.0,8.3,...,UNABLE TO OBTAIN,21.333333,18,119.0,0.6,1,0.0,0,13.6,103.5
0,10844468,20367176,31505387,False,23.0,54,359.0,10.0,22.0,8.4,...,WHITE,20.500000,27,133.5,1.3,3,0.0,0,21.4,NaN
0,16662264,20376296,39256900,False,NaN,50,NaN,NaN,NaN,NaN,...,WHITE,14.500000,27,106.0,NaN,2,0.0,0,NaN,53.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,12019706,29822163,32302734,False,25.0,67,735.0,17.0,36.0,8.1,...,UNKNOWN,22.500000,44,106.0,2.7,3,0.0,0,22.2,NaN
0,18624988,29915993,31559071,True,19.0,82,508.0,19.0,54.0,7.5,...,WHITE,24.250000,63,111.5,2.3,7,0.0,0,26.4,120.3
0,18322831,29964109,30754159,True,NaN,66,NaN,NaN,NaN,NaN,...,WHITE,NaN,42,NaN,NaN,5,0.0,0,NaN,NaN
0,14320833,29970786,34923650,True,NaN,64,NaN,NaN,NaN,NaN,...,WHITE,15.000000,41,132.5,NaN,2,0.0,0,NaN,81.7
